<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/AGENTIC-MODULE4-Sensor-Pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Sensor Model Pre-Training System - LSTM Autoencoder Only
========================================================

This script handles the initial training of sensor models using your pre-processed dataset.
Uses LSTM Autoencoder for anomaly detection. Models are trained once and saved to disk
for use in the production agent system.

Dataset format: [num_samples, window_length, num_sensors] - already scaled and sequenced.

Usage:
    python sensor_pretraining.py --data_path /path/to/dataset.npy --models_dir ./trained_models
"""

import numpy as np
import pandas as pd
import pickle
import os
import argparse
from datetime import datetime
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

# Deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential, Model
    from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Input
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
    KERAS_AVAILABLE = True
except ImportError:
    print("⚠️ TensorFlow not available. Install with: pip install tensorflow")
    KERAS_AVAILABLE = False

from sklearn.metrics import mean_squared_error


class SensorModelTrainer:
    """
    Handles training and saving of LSTM Autoencoder models for individual sensors.
    """

    def __init__(self,
                 window_length: int = 50,
                 latent_dim: int = 32,
                 epochs: int = 100,
                 batch_size: int = 32):

        self.window_length = window_length
        self.latent_dim = latent_dim
        self.epochs = epochs
        self.batch_size = batch_size

        if not KERAS_AVAILABLE:
            raise ImportError("TensorFlow required for model training")

    def build_lstm_autoencoder(self) -> Model:
        """Build LSTM Autoencoder for anomaly detection."""

        # Encoder
        inputs = Input(shape=(self.window_length, 1), name='encoder_input')
        encoded = LSTM(self.latent_dim, activation='relu', return_sequences=False, name='encoder_lstm')(inputs)

        # Decoder
        decoded = RepeatVector(self.window_length, name='repeat_vector')(encoded)
        decoded = LSTM(self.latent_dim, activation='relu', return_sequences=True, name='decoder_lstm')(decoded)
        outputs = TimeDistributed(Dense(1, activation='linear'), name='decoder_output')(decoded)

        # Create model
        model = Model(inputs, outputs, name='sensor_lstm_autoencoder')
        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )

        return model

    def train_sensor_model(self, sensor_data: np.ndarray, sensor_id: int) -> Tuple[Model, Dict]:
        """
        Train LSTM Autoencoder for a single sensor using pre-prepared sequences.

        Args:
            sensor_data: Pre-scaled sequences [num_samples, window_length] for this sensor
            sensor_id: Sensor identifier

        Returns:
            Trained model and training history
        """

        print(f"Training LSTM Autoencoder for sensor {sensor_id}...")
        print(f"  Data shape: {sensor_data.shape}")
        print(f"  Total sequences: {len(sensor_data)}")

        if len(sensor_data) < 100:
            raise ValueError(f"Insufficient sequences ({len(sensor_data)}) for training. Need at least 100.")

        # Data is already in correct format for LSTM: [num_samples, window_length, features]
        # If features=1 (univariate), we might need to add the feature dimension
        if sensor_data.ndim == 2:
            # Add feature dimension: [num_samples, window_length] -> [num_samples, window_length, 1]
            X = sensor_data.reshape(len(sensor_data), self.window_length, 1)
            print(f"  Added feature dimension: {sensor_data.shape} -> {X.shape}")
        else:
            # Data already has feature dimension
            X = sensor_data
            print(f"  Data already in LSTM format: {X.shape}")

        # Verify final shape
        expected_shape = (len(sensor_data), self.window_length, 1)
        if X.shape != expected_shape:
            raise ValueError(f"Expected shape {expected_shape}, got {X.shape}")

        # Split data into train/validation/test
        # 70% train, 15% validation, 15% test
        n_samples = len(X)
        n_train = int(0.7 * n_samples)
        n_val = int(0.15 * n_samples)

        # Shuffle indices to randomize splits
        indices = np.random.RandomState(42).permutation(n_samples)

        train_idx = indices[:n_train]
        val_idx = indices[n_train:n_train + n_val]
        test_idx = indices[n_train + n_val:]

        X_train = X[train_idx]
        X_val = X[val_idx]
        X_test = X[test_idx]

        print(f"  Train sequences: {len(X_train)} | shape: {X_train.shape}")
        print(f"  Validation sequences: {len(X_val)} | shape: {X_val.shape}")
        print(f"  Test sequences: {len(X_test)} | shape: {X_test.shape}")

        # Build LSTM Autoencoder
        model = self.build_lstm_autoencoder()
        print(f"  Model: LSTM Autoencoder with {model.count_params():,} parameters")

        # Callbacks
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
        ]

        # Train model
        start_time = datetime.now()

        print("  Starting training...")
        history = model.fit(
            X_train, X_train,  # Autoencoder: input = output
            epochs=self.epochs,
            batch_size=self.batch_size,
            validation_data=(X_val, X_val),
            callbacks=callbacks,
            verbose=1
        )

        training_time = datetime.now() - start_time
        print(f"  Training completed in {training_time}")

        # Evaluate on test set
        print("  Evaluating on test set...")
        test_loss = model.evaluate(X_test, X_test, verbose=0)
        test_mse = test_loss[0] if isinstance(test_loss, list) else test_loss
        test_mae = test_loss[1] if isinstance(test_loss, list) and len(test_loss) > 1 else 0.0

        print(f"  Test MSE: {test_mse:.6f}")
        if test_mae > 0:
            print(f"  Test MAE: {test_mae:.6f}")

        # Compute baseline errors for anomaly detection using validation set
        print("  Computing baseline error statistics...")
        baseline_errors = []

        # Process validation set in batches to compute reconstruction errors
        for i in range(0, len(X_val), self.batch_size):
            batch = X_val[i:i + self.batch_size]
            predictions = model.predict(batch, verbose=0)

            # Compute MSE for each sequence in the batch
            for j, pred in enumerate(predictions):
                error = mean_squared_error(batch[j].flatten(), pred.flatten())
                baseline_errors.append(error)

        # Statistical analysis of baseline errors
        baseline_stats = {
            'mean': float(np.mean(baseline_errors)),
            'std': float(np.std(baseline_errors)) + 1e-8,  # Add small epsilon
            'median': float(np.median(baseline_errors)),
            'q75': float(np.percentile(baseline_errors, 75)),
            'q90': float(np.percentile(baseline_errors, 90)),
            'q95': float(np.percentile(baseline_errors, 95)),
            'q99': float(np.percentile(baseline_errors, 99)),
            'min': float(np.min(baseline_errors)),
            'max': float(np.max(baseline_errors)),
            'count': len(baseline_errors)
        }

        # Prepare training information
        training_info = {
            'sensor_id': sensor_id,
            'model_type': 'lstm_autoencoder',
            'architecture': {
                'window_length': self.window_length,
                'latent_dim': self.latent_dim,
                'total_params': int(model.count_params())
            },
            'training_data': {
                'total_sequences': len(sensor_data),
                'training_sequences': len(X_train),
                'validation_sequences': len(X_val),
                'test_sequences': len(X_test),
                'input_shape': list(X.shape)
            },
            'training_process': {
                'epochs_requested': self.epochs,
                'epochs_trained': len(history.history['loss']),
                'batch_size': self.batch_size,
                'training_time': str(training_time),
                'early_stopping': len(history.history['loss']) < self.epochs
            },
            'performance': {
                'final_train_loss': float(history.history['loss'][-1]),
                'final_val_loss': float(history.history['val_loss'][-1]),
                'best_val_loss': float(min(history.history['val_loss'])),
                'test_mse': float(test_mse),
                'test_mae': float(test_mae) if test_mae > 0 else None
            },
            'baseline_errors': baseline_errors[-100:],  # Store last 100 for drift detection
            'baseline_stats': baseline_stats,
            'trained_at': datetime.now(),
            'tensorflow_version': tf.__version__ if KERAS_AVAILABLE else None
        }

        # Training summary
        print(f"  ✅ Training successful!")
        print(f"     Epochs trained: {training_info['training_process']['epochs_trained']}")
        print(f"     Final train loss: {training_info['performance']['final_train_loss']:.6f}")
        print(f"     Final val loss: {training_info['performance']['final_val_loss']:.6f}")
        print(f"     Test MSE: {training_info['performance']['test_mse']:.6f}")
        print(f"     Baseline error: {baseline_stats['mean']:.6f} ± {baseline_stats['std']:.6f}")
        print(f"     Error range: [{baseline_stats['min']:.6f}, {baseline_stats['max']:.6f}]")

        return model, training_info

    def save_model(self, model: Model, training_info: Dict, models_dir: str) -> Tuple[str, str]:
        """Save trained model and metadata."""

        sensor_id = training_info['sensor_id']
        os.makedirs(models_dir, exist_ok=True)

        # Save model in Keras format
        model_path = os.path.join(models_dir, f"sensor_{sensor_id}_model.h5")
        model.save(model_path)

        # Save metadata
        metadata_path = os.path.join(models_dir, f"sensor_{sensor_id}_metadata.pkl")
        with open(metadata_path, 'wb') as f:
            pickle.dump(training_info, f)

        print(f"  💾 Saved model: {model_path}")
        print(f"  💾 Saved metadata: {metadata_path}")

        return model_path, metadata_path


def load_your_dataset(data_path: str) -> Tuple[np.ndarray, np.ndarray]:
    """
    Load your pre-processed dataset.

    Args:
        data_path: Path to your dataset

    Returns:
        Tuple of (training_data, holdout_data)
        - training_data: [num_samples, window_length, num_sensors] for model training
        - holdout_data: [1000, window_length, num_sensors] for streaming simulation
    """

    print(f"📂 Loading pre-processed dataset from: {data_path}")

    if data_path.endswith('.npy'):
        # Numpy array format
        print("  Format: Numpy array (.npy)")
        data = np.load(data_path)

    elif data_path.endswith('.npz'):
        # Compressed numpy format
        print("  Format: Compressed numpy (.npz)")
        data_file = np.load(data_path)
        # Assume main data is stored with key 'data' or use first key
        if 'data' in data_file:
            data = data_file['data']
        else:
            data = data_file[list(data_file.keys())[0]]

    elif data_path.endswith('.h5') or data_path.endswith('.hdf5'):
        # HDF5 format
        print("  Format: HDF5 file")
        import h5py

        with h5py.File(data_path, 'r') as f:
            # Assume main data is stored with key 'data' or use first key
            if 'data' in f:
                data = f['data'][:]
            else:
                data = f[list(f.keys())[0]][:]

    elif data_path.endswith('.pkl') or data_path.endswith('.pickle'):
        # Pickle format
        print("  Format: Pickle file")
        import pickle
        with open(data_path, 'rb') as f:
            data = pickle.load(f)
            # If pickle contains dict, extract the array
            if isinstance(data, dict):
                if 'data' in data:
                    data = data['data']
                else:
                    data = list(data.values())[0]

    else:
        raise ValueError(f"Unsupported data format: {data_path}. Supported: .npy, .npz, .h5, .hdf5, .pkl")

    # Validate data format
    if not isinstance(data, np.ndarray):
        raise ValueError(f"Data must be numpy array, got {type(data)}")

    if data.ndim != 3:
        raise ValueError(f"Data must be 3D [num_samples, window_length, num_sensors], got {data.ndim}D: {data.shape}")

    num_samples, window_length, num_sensors = data.shape
    print(f"  📊 Dataset shape: {data.shape}")
    print(f"     Total samples: {num_samples:,}")
    print(f"     Window length: {window_length}")
    print(f"     Number of sensors: {num_sensors}")

    # Check if data appears scaled
    data_min, data_max = np.min(data), np.max(data)
    print(f"  📈 Data range: [{data_min:.3f}, {data_max:.3f}]")

    if -10 <= data_min and data_max <= 10:
        print("  ✅ Data appears to be pre-scaled")
    else:
        print("  ⚠️ Data range seems large - verify it's properly scaled")

    # Check for invalid values
    invalid_count = np.sum(~np.isfinite(data))
    if invalid_count > 0:
        print(f"  ⚠️ Found {invalid_count} invalid values (NaN/Inf) - will be handled during training")

    # Split into training and holdout
    if num_samples <= 1000:
        raise ValueError(f"Dataset too small ({num_samples} samples). Need more than 1000 samples.")

    holdout_data = data[-1000:].copy()  # Last 1000 samples
    training_data = data[:-1000].copy()  # Everything except last 1000

    print(f"  📊 Data split:")
    print(f"     Training: {training_data.shape[0]:,} samples")
    print(f"     Holdout: {holdout_data.shape[0]:,} samples")

    return training_data, holdout_data


def validate_dataset(dataset: np.ndarray, window_length: int) -> np.ndarray:
    """
    Validate the pre-processed dataset.

    Args:
        dataset: Pre-processed dataset [num_samples, window_length, num_sensors]
        window_length: Expected window length

    Returns:
        Validated dataset
    """

    print("🔍 Validating dataset...")

    num_samples, actual_window_length, num_sensors = dataset.shape

    # Check window length
    if actual_window_length != window_length:
        raise ValueError(f"Window length mismatch: expected {window_length}, got {actual_window_length}")

    # Check for invalid values
    invalid_mask = ~np.isfinite(dataset)
    invalid_count = np.sum(invalid_mask)

    if invalid_count > 0:
        invalid_ratio = invalid_count / dataset.size
        print(f"  ⚠️ Found {invalid_count:,} invalid values ({invalid_ratio:.2%} of total)")

        if invalid_ratio > 0.1:
            raise ValueError(f"Too many invalid values ({invalid_ratio:.1%}). Check data quality.")

        # Replace invalid values with sensor means
        print("  🔧 Replacing invalid values with sensor means...")
        for sensor_id in range(num_sensors):
            sensor_data = dataset[:, :, sensor_id]
            valid_data = sensor_data[np.isfinite(sensor_data)]
            if len(valid_data) > 0:
                mean_val = np.mean(valid_data)
                sensor_mask = invalid_mask[:, :, sensor_id]
                dataset[:, :, sensor_id][sensor_mask] = mean_val

    # Statistical validation per sensor
    print(f"  📊 Per-sensor statistics:")
    for sensor_id in range(num_sensors):
        sensor_data = dataset[:, :, sensor_id]

        mean_val = np.mean(sensor_data)
        std_val = np.std(sensor_data)
        min_val = np.min(sensor_data)
        max_val = np.max(sensor_data)

        print(f"    Sensor {sensor_id}: μ={mean_val:.3f}, σ={std_val:.3f}, "
              f"range=[{min_val:.3f}, {max_val:.3f}]")

        # Check for constant values
        if std_val < 1e-6:
            print(f"    ⚠️ Sensor {sensor_id}: Nearly constant values (σ={std_val:.2e})")

    print(f"✅ Dataset validation completed")
    return dataset


def main():
    """Main training pipeline for pre-processed dataset."""

    parser = argparse.ArgumentParser(description="Train LSTM Autoencoder models on pre-processed dataset")
    parser.add_argument('--data_path', type=str, required=True,
                       help='Path to pre-processed dataset [num_samples, window_length, num_sensors]')
    parser.add_argument('--models_dir', type=str, default='./trained_models',
                       help='Directory to save models')
    parser.add_argument('--holdout_dir', type=str, default='./holdout_data',
                       help='Directory to save holdout data for streaming simulation')
    parser.add_argument('--window_length', type=int, default=50,
                       help='Expected sequence window length')
    parser.add_argument('--latent_dim', type=int, default=32,
                       help='LSTM latent dimension')
    parser.add_argument('--epochs', type=int, default=100, help='Training epochs')
    parser.add_argument('--batch_size', type=int, default=32, help='Batch size')
    parser.add_argument('--sensors', type=str, default=None,
                       help='Comma-separated sensor IDs to train (default: all)')

    args = parser.parse_args()

    print("🚀 SENSOR MODEL PRE-TRAINING SYSTEM")
    print("=" * 50)
    print(f"📂 Data path: {args.data_path}")
    print(f"💾 Models directory: {args.models_dir}")
    print(f"📦 Holdout directory: {args.holdout_dir}")
    print(f"📏 Window length: {args.window_length}")
    print(f"🧠 Model: LSTM Autoencoder (latent_dim={args.latent_dim})")
    print(f"🔄 Epochs: {args.epochs}")
    print(f"📦 Batch size: {args.batch_size}")
    print()

    # Load and split dataset
    training_data, holdout_data = load_your_dataset(args.data_path)

    # Validate dataset
    training_data = validate_dataset(training_data, args.window_length)

    # Save holdout data for streaming simulation
    os.makedirs(args.holdout_dir, exist_ok=True)
    holdout_path = os.path.join(args.holdout_dir, 'holdout_data.npy')
    np.save(holdout_path, holdout_data)
    print(f"💾 Saved holdout data to: {holdout_path}")
    print(f"   Holdout shape: {holdout_data.shape}")

    # Get dataset dimensions
    num_samples, window_length, num_sensors = training_data.shape

    # Filter sensors if specified
    sensor_list = list(range(num_sensors))
    if args.sensors:
        requested_sensors = [int(x.strip()) for x in args.sensors.split(',')]
        sensor_list = [sid for sid in requested_sensors if sid < num_sensors]
        print(f"🎯 Training only sensors: {sensor_list}")

    # Initialize trainer
    trainer = SensorModelTrainer(
        window_length=window_length,
        latent_dim=args.latent_dim,
        epochs=args.epochs,
        batch_size=args.batch_size
    )

    # Train models for each sensor
    print(f"\n🏋️ TRAINING {len(sensor_list)} LSTM AUTOENCODER MODELS")
    print("=" * 50)

    training_results = {}
    successful_training = 0

    for sensor_id in sensor_list:
        try:
            print(f"\n🎯 SENSOR {sensor_id}")
            print("-" * 30)

            # Extract data for this sensor: [num_samples, window_length]
            sensor_sequences = training_data[:, :, sensor_id]

            # Train model
            model, training_info = trainer.train_sensor_model(sensor_sequences, sensor_id)

            # Save model
            model_path, metadata_path = trainer.save_model(model, training_info, args.models_dir)

            training_results[sensor_id] = {
                'success': True,
                'model_path': model_path,
                'metadata_path': metadata_path,
                'training_info': training_info
            }

            successful_training += 1

        except Exception as e:
            print(f"  ❌ Training failed: {str(e)}")
            import traceback
            print(f"  📋 Error details: {traceback.format_exc()}")

            training_results[sensor_id] = {
                'success': False,
                'error': str(e)
            }

    # Final summary
    print(f"\n📊 TRAINING SUMMARY")
    print("=" * 50)
    print(f"✅ Successful: {successful_training}/{len(sensor_list)} sensors")
    print(f"💾 Models saved to: {args.models_dir}")
    print(f"📦 Holdout data saved to: {holdout_path}")

    if successful_training > 0:
        print(f"\n🏆 TRAINED SENSORS:")
        for sensor_id, result in training_results.items():
            if result['success']:
                info = result['training_info']
                perf = info['performance']
                print(f"  Sensor {sensor_id}: {info['training_process']['epochs_trained']} epochs, "
                      f"test MSE: {perf['test_mse']:.6f}, "
                      f"baseline error: {info['baseline_stats']['mean']:.6f}")

    failed_sensors = [sid for sid, result in training_results.items() if not result['success']]
    if failed_sensors:
        print(f"\n❌ FAILED SENSORS: {failed_sensors}")
        print("   Check error messages above for details")

    # Save comprehensive training summary
    summary_path = os.path.join(args.models_dir, 'training_summary.pkl')
    training_summary = {
        'training_results': training_results,
        'config': vars(args),
        'dataset_info': {
            'original_shape': training_data.shape,
            'holdout_shape': holdout_data.shape,
            'num_sensors': num_sensors,
            'window_length': window_length,
            'total_training_samples': num_samples
        },
        'model_info': {
            'model_type': 'lstm_autoencoder',
            'latent_dim': args.latent_dim,
            'architecture': 'Encoder-Decoder with RepeatVector'
        },
        'timestamp': datetime.now(),
        'successful_sensors': successful_training,
        'failed_sensors': failed_sensors,
        'holdout_data_path': holdout_path,
        'tensorflow_version': tf.__version__ if KERAS_AVAILABLE else None
    }

    with open(summary_path, 'wb') as f:
        pickle.dump(training_summary, f)

    print(f"\n💾 Training summary saved to: {summary_path}")
    print(f"✅ PRE-TRAINING COMPLETED!")

    # Instructions for next steps
    print(f"\n📋 NEXT STEPS:")
    print(f"  1. Use trained models: {args.models_dir}")
    print(f"  2. Use holdout data for streaming: {holdout_path}")
    print(f"  3. Run production system with:")
    print(f"     python production_agent_system.py --models_dir {args.models_dir}")


if __name__ == "__main__":
    main()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')